In [12]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [2]:
import datetime
import pprint
import pyucrio

rio = pyucrio.PyUCRio()

/opt/miniconda/envs/pyucrio39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Download data

In [3]:
# list available datasets
rio.data.list_datasets_in_table()

Name                                     Provider   Level   DOI Details                                                             Short Description                                                   
NORSTAR_RIOMETER_K0_TXT                  UCalgary   L0      Coming soon                                                             NORSTAR Single Frequency Riometers K0 raw data, in ASCII format     
NORSTAR_RIOMETER_K2_TXT                  UCalgary   L2      Coming soon                                                             NORSTAR Single Frequency Riometers K2 baselined data, in ASCII      
                                                                                                                                    format                                                              
REGO_CALIBRATION_FLATFIELD_IDLSAV        UCalgary   L3      https://commons.datacite.org/doi.org/10.11575/z7x6-5c42                 REGO All Sky Imagers Flatfield calibration data (IDL save format

In [4]:
# download a few days of NORSTAR Riometer data from Churchill
dataset_name = "NORSTAR_RIOMETER_K0_TXT"
start_dt = datetime.datetime(2021, 11, 3, 0, 0)
end_dt = datetime.datetime(2021, 11, 5, 23, 59)
site_uid = "chur"
r = rio.data.ucalgary.download(dataset_name, start_dt, end_dt, site_uid=site_uid)

In [5]:
# view information about the downloaded data
print(r)
print()

pprint.pprint(r.__dict__)

FileDownloadResult(filenames=[PosixPath('/home/darrenc/pyucrio_data/NORSTAR_RIOMETER_K0_TXT/2021/11/03/norstar_k0_rio-chur_20211103_v01.txt'), PosixPath('/home/darrenc/pyucrio_data/NORSTAR_RIOMETER_K0_TXT/2021/11/04/norstar_k0_rio-chur_20211104_v01.txt'), PosixPath('/home/darrenc/pyucrio_data/NORSTAR_RIOMETER_K0_TXT/2021/11/05/norstar_k0_rio-chur_20211105_v01.txt')], count=3, total_bytes=1609116, output_root_path='/home/darrenc/pyucrio_data/NORSTAR_RIOMETER_K0_TXT', dataset=Dataset(name=NORSTAR_RIOMETER_K0_TXT, short_description='NORSTAR Single Frequency Riometers K0 raw data, in ASCII format', provider='UCalgary', level='L0', doi_details='Coming soon', ...))

{'count': 3,
 'dataset': Dataset(name=NORSTAR_RIOMETER_K0_TXT, short_description='NORSTAR Single Frequency Riometers K0 raw data, in ASCII format', provider='UCalgary', level='L0', doi_details='Coming soon', ...),
 'filenames': [PosixPath('/home/darrenc/pyucrio_data/NORSTAR_RIOMETER_K0_TXT/2021/11/03/norstar_k0_rio-chur_20211103_

In [6]:
# download a couple days of data from all NORSTAR riometer sites
dataset_name = "NORSTAR_RIOMETER_K0_TXT"
start_dt = datetime.datetime(2021, 11, 3, 0, 0)
end_dt = datetime.datetime(2021, 11, 5, 23, 59)
_ = rio.data.ucalgary.download(dataset_name, start_dt, end_dt)

# Read data

The data reading routines take in a list of filenames on your computer. The list of files is returned in a `download()` call, but can also be created using `glob` or similar.

There are two methods that can be used for reading data:

1) using the generic method
2) using a specific dataset read function

The generic method is the recommended way, but if more control is wanted, you can use the specific read functions directly. The generic method simply uses the dataset name to determine which specific read function to use.


In [7]:
# let's show the generic method first, since it is the easiest way
data = rio.data.ucalgary.read(r.dataset, r.filenames, n_parallel=2)

print(data)
print()

data.pretty_print()

Data(data=[3 RiometerData objects], timestamp=[3 datetimes], metadata=[3 dictionaries], problematic_files=[], calibrated_data=None, dataset=Dataset(name=NORSTAR_RIOMETER_K0_TXT, short_description='NORSTAR Single Fre...))

Data:
  data                  : [3 RiometerData objects]
  timestamp             : [3 datetimes]
  metadata              : [3 dictionaries]
  problematic_files     : []
  calibrated_data       : None
  dataset               : Dataset(name=NORSTAR_RIOMETER_K0_TXT, short_description='NORSTAR Single Fre...)


In [8]:
# since we know we're reading in NORSTAR riometer k0 data, we can also use the specific read routine. In most
# circumstances, this method isn't necessary to use.
data = rio.data.ucalgary.readers.read_norstar_riometer(r.filenames, n_parallel=2, dataset=r.dataset)

print(data)
print()

data.pretty_print()

Data(data=[3 RiometerData objects], timestamp=[3 datetimes], metadata=[3 dictionaries], problematic_files=[], calibrated_data=None, dataset=Dataset(name=NORSTAR_RIOMETER_K0_TXT, short_description='NORSTAR Single Fre...))

Data:
  data                  : [3 RiometerData objects]
  timestamp             : [3 datetimes]
  metadata              : [3 dictionaries]
  problematic_files     : []
  calibrated_data       : None
  dataset               : Dataset(name=NORSTAR_RIOMETER_K0_TXT, short_description='NORSTAR Single Fre...)


# Managing downloaded data

In [10]:
# to view the amount of data that is currently downloaded, do the following
rio.show_data_usage()

Dataset name              Size   
NORSTAR_RIOMETER_K0_TXT   23.1 MB
SWAN_HSR_K0_H5            18.7 MB

Total size: 41.7 MB


In [11]:
# alternatively, you can get the data usage information returned as a dictionary
data_usage_dict = rio.show_data_usage(return_dict=True)

In [12]:
# to clean up all data we've downloaded, you can do the following.
# 
# alternatively, you can manually delete data yourself
rio.purge_download_output_root_path()